### Imports

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import jax.numpy as jnp
from jax.numpy import resize
import numpy as np
import cv2
import os
import zipfile
from google.colab.patches import cv2_imshow
from google.colab import files
from ipywidgets import interact, interactive, interact_manual
from ipywidgets import interactive, IntSlider, Button, VBox, HBox
from IPython.display import display
from IPython.display import clear_output


## Mask Pipeline: Edge Detection

In [ ]:
mask = [cv2.imread(f"mask/rust_squared_constant_{i}.png") for i in range(0,514)]     # loading all the rust diseases data and storing it on rust_data variable
mask_gray = [cv2.imread(f"mask/rust_squared_constant_{i}.png", cv2.IMREAD_GRAYSCALE) for i in range(0,514)]     # loading all the rust diseases data and storing it on rust_data variable

mask_rust_data_color = [cv2.imread(f"rust_pre_processing/rust_squared_constant_{i}.png") for i in range(0,514)]     # loading all the rust diseases data and storing it on rust_data variable
mask_rust_data_gray = [cv2.imread(f"rust_pre_processing/rust_squared_constant_{i}.png", cv2.IMREAD_GRAYSCALE) for i in range(0,514)]     # loading all the rust diseases data and storing it on rust_data variable


In [ ]:
# Border Detector Function
def border_detector(image_index, canny_inf, canny_sup, kernel_sobel, filter):
    global img_indx
    global new_image
    global new_image_closed
    global image_mult
    global filter_name

    sobel_kernel = np.ones((kernel_sobel, kernel_sobel), np.uint8)

    img_indx = image_index
    image = mask_rust_data_color[image_index]

    # Converting image to gray scale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Canny Filtering
    if filter == "canny":
      gray_image_8u = cv2.convertScaleAbs(gray_image)
      new_image_closed = cv2.Canny(gray_image_8u, canny_inf, canny_sup)

    # Sobel Filtering
    elif filter == "sobel":
      sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=kernel_sobel)
      sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=kernel_sobel)
      magnitude = cv2.magnitude(sobel_x, sobel_y)
      magnitude = np.uint8(np.absolute(magnitude))
      new_image_closed = magnitude

    # Laplacian Filtering
    elif filter == "laplacian":
      laplacian = cv2.Laplacian(gray_image, cv2.CV_64F)
      laplacian = cv2.convertScaleAbs(laplacian)
      new_image_closed =  laplacian


    cv2_imshow(new_image_closed)

filters = ["canny", "laplacian", "sobel"]       # setting possible filters
button = Button(description="Apply Changes")    # creating the button

# Creating IntSlider for dynamic variables
image_slider = IntSlider(value=71, min=1, max=513, step=1, description='Image')
canny_inf = IntSlider(value=71, min=1, max=255, step=1, description='canny_inf')
canny_sup = IntSlider(value=71, min=1, max=255, step=1, description='canny_sup')
dilated =  IntSlider(value=71, min=1, max=31, step=2, description='kernel_sobel')

# Interactive Widget
interactive_widget = interactive(border_detector,
    image_index = image_slider,
    canny_inf = canny_inf,
    canny_sup = canny_sup,
    kernel_sobel = dilated,
    filter=filters
)
display(interactive_widget)

# Save Button: Function
def on_button_click(b):
    if image_slider.value < image_slider.max:
        print(f"Image saved, index: {img_indx}")
        mask_rust_data_color[img_indx] = new_image_closed
    else: pass

# Display
button.on_click(on_button_click)
display(VBox([button]))


interactive(children=(IntSlider(value=71, description='Image', max=513, min=1), IntSlider(value=71, descriptio…

## Mask Pipeline: Mapping Duplicated Images

In [ ]:
import pandas as pd
import re

global healthy_duplicated
global mosaic_duplicated
global redrot_duplicated
global rust_duplicated
global yellow_duplicated

def extract_feature(path):
    match = re.search(r'(healthy|mosaic|rust|yellow)_squared_(constant|edge)_(\d+)', path)
    if match:
        return f"{match.group(1)}"
    return None

def extract_duplicate_index(path):
    match = re.search(r'(healthy|mosaic|rust|yellow)_squared_(constant|edge)_(\d+)', path)
    if match:
        return int(match.group(3))  # Convertendo para int
    return None

# reading csv file
duplicate = pd.read_csv("duplicates_mapping.csv")

# creating new columns
duplicate['Feature'] = duplicate['DuplicatedImage'].apply(extract_feature)
duplicate['DuplicatedIndex'] = duplicate['DuplicatedImage'].apply(extract_duplicate_index)


# duplicated values
healthy_duplicated = duplicate.loc[duplicate["Feature"] == "healthy"]["DuplicatedIndex"].unique()
mosaic_duplicated = duplicate.loc[duplicate["Feature"] == "mosaic"]["DuplicatedIndex"].unique()
redrot_duplicated = duplicate.loc[duplicate["Feature"] == "redrot"]["DuplicatedIndex"].unique()
yellow_duplicated = duplicate.loc[duplicate["Feature"] == "yellow"]["DuplicatedIndex"].unique()

rust_duplicated = duplicate.loc[duplicate["Feature"] == "rust"]["DuplicatedIndex"].unique()
rust_duplicated = list(rust_duplicated)
rust_duplicated.append(194)


duplicate.head(3)



,DuplicatedImage,OriginalImage,Feature,DuplicatedIndex
0,C:\Users\Karina Rocha\SugarcaneLeafDisease\dat...,C:\Users\Karina Rocha\SugarcaneLeafDisease\dat...,healthy,188
1,C:\Users\Karina Rocha\SugarcaneLeafDisease\dat...,C:\Users\Karina Rocha\SugarcaneLeafDisease\dat...,healthy,197
2,C:\Users\Karina Rocha\SugarcaneLeafDisease\dat...,C:\Users\Karina Rocha\SugarcaneLeafDisease\dat...,healthy,200


In [21]:
yellow_duplicated

array([248, 249, 267, 268, 269, 284, 290, 386, 441, 448,  84])

## Mask Pipeline: Smoothing

In [ ]:
global yellow
global yellow_mask
global rust
global rust_mask
global mosaic
global mosaic_mask
global redrot
global redrot_mask
global healthy
global healthy_mask

# Resized Dataset
yellow = [cv2.imread(f"yellow/yellow_squared_constant_{i}.png") for i in range(0,505)]   # loading yellow data
rust = [cv2.imread(f"rust/rust_squared_constant_{i}.png") for i in range(0,514)]         # loading rust data
redrot = [cv2.imread(f"redrot/redrot_squared_constant_{i}.png") for i in range(0,518)]                 # loading redrot data
mosaic = [cv2.imread(f"mosaic/mosaic_squared_constant_{i}.png") for i in range(0,462)]   # loading mosaic data
healthy = [cv2.imread(f"healthy/healthy_squared_constant_{i}.png") for i in range(0,522)]              # loading healthy data

# Raw masks provided by border_detector or photoshop's background removal
yellow_mask = [cv2.imread(f"yellow_mask/yellow_squared_constant_{i}.png") for i in range(0,505)]
rust_mask = [cv2.imread(f"rust_mask/rust_squared_constant_{i}.png") for i in range(0,514)]
redrot_mask = [cv2.imread(f"redrot_mask/redrot_squared_constant_{i}.png") for i in range(0,518)]
mosaic_mask = [cv2.imread(f"mosaic_mask/mosaic_squared_constant_{i}.png") for i in range(0,462)]
healthy_mask = [cv2.imread(f"healthy_mask/healthy_squared_constant_{i}.png") for i in range(0,522)]

In [22]:
class Mask():
    def __init__(self):
        pass

    def view(self, dataset: str, mask_index: int):
        self.mask_index = mask_index
        self.dataset = dataset

        # Duplicated values calculated on the previous line
        duplicated = {
            "yellow": yellow_duplicated,
            "rust": rust_duplicated,
            "mosaic": mosaic_duplicated,
            "redrot": redrot_duplicated,
            "healthy": healthy_duplicated
        }

        # Datasets: features
        datasets = {
            "yellow": yellow,
            "rust": rust,
            "mosaic": mosaic,
            "redrot": redrot,
            "healthy": healthy
        }

        # Masks: pseudo-masks obtained from border_detector and Photoshop's background removal
        masks = {
            "yellow": yellow_mask,
            "rust": rust_mask,
            "mosaic": mosaic_mask,
            "redrot": redrot_mask,
            "healthy": healthy_mask
        }

        # Obtain dataset and mask properly based on input from the interactive window
        self.disease = datasets.get(dataset, None)
        self.mask = masks.get(dataset, None)

        # Limit check
        if mask_index >= len(self.disease):
            mask_index = len(self.disease)

        # Getting the mask selected based on the interactive view
        mask_selected = self.mask[mask_index]

        self.duplicated = duplicated[self.dataset]
        self.pipeline(mask_selected)
        self.plot()

    def pipeline(self, mask):
        # Converting to grayscale
        image_gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

        # Thresholding
        _, self.binary_mask = cv2.threshold(image_gray, 0, 255, cv2.THRESH_BINARY)

        # Median filtering
        self.mask_median = cv2.medianBlur(self.binary_mask, 7)

        # Gaussian filtering
        self.mask_gaussian = cv2.GaussianBlur(self.mask_median, (7, 7), 0)

        # Opening
        self.opened = cv2.morphologyEx(self.mask_gaussian, cv2.MORPH_OPEN,
                                   cv2.getStructuringElement(cv2.MORPH_RECT, (20, 20)))

        # Converting eroded mask back to [0,1]
        opened_bin = self.opened // 255  # This will convert 255 to 1, and 0 to 0

        # Expanding it through 3 channels
        opened_3channels = cv2.merge([opened_bin, opened_bin, opened_bin])  # Merging the 3 channels with the same mask

        disease_selected = self.disease[self.mask_index]

        # Multiply -> image ⊙ mask
        self.result = cv2.multiply(disease_selected, opened_3channels.astype(np.uint8))

        return self.result

    def zip(self, *args):
        output_dir = f'/content/{self.dataset}_temp'
        os.makedirs(output_dir, exist_ok=True)

        image_paths = []  # paths to the images
        count = 0         # counter variable to enumerate the images

        # Iterating over images and applying the pipeline to each one
        for i, img in enumerate(self.disease):
            self.mask_index = i

            # Duplicated image found
            if i in list(self.duplicated):
                pass

            # Not duplicated
            else:
                mask_selected = self.mask[self.mask_index]
                processed_img = self.pipeline(mask_selected)  # Processing the image with the pipeline

                # Defining the image path and saving
                image_path = os.path.join(output_dir, f"{self.dataset}_mask_{count}.png")
                cv2.imwrite(image_path, processed_img)  # Saving the image
                image_paths.append(image_path)  # Saving the image path
                count += 1

        # Creating the ZIP file
        zip_file = f'/content/{self.dataset}_segmentation.zip'
        with zipfile.ZipFile(zip_file, 'w') as zipf:
            for img_path in image_paths:
                zipf.write(img_path, os.path.basename(img_path))  # Adding the image to the zip

        # Verifying if the ZIP file was created
        os.path.exists(zip_file)

        # Downloading the ZIP file
        files.download(zip_file)

        print("zipped")

    def plot(self):
        # Display results side by side
        clear_output(wait=True)
        fig, axes = plt.subplots(1, 5, figsize=(12, 6))  # 1 row, 5 columns

        axes[0].imshow(self.binary_mask, cmap='gray')
        axes[0].set_title('(1) Raw Mask')
        axes[0].axis('off')

        axes[1].imshow(self.mask_median, cmap='gray')
        axes[1].set_title('(2) Median 7x7')
        axes[1].axis('off')

        axes[2].imshow(self.mask_gaussian, cmap='gray')
        axes[2].set_title('(3) Gaussian 7x7')
        axes[2].axis('off')

        axes[3].imshow(self.opened)
        axes[3].set_title('(4) Opening 20x20')
        axes[3].axis('off')

        axes[4].imshow(self.result)
        axes[4].set_title('(5) Result')
        axes[4].axis('off')

        plt.tight_layout()
        plt.show()


In [ ]:
# Creating IntSlider for dynamic variables
leaf_disease = ["healthy", "redrot", "mosaic", "rust", "yellow"]
index = IntSlider(value=71, min=0, max=522, step=1, description='canny_inf')

mask = Mask()

# Interactive Widget
interactive_widget = interactive(mask.view,
    dataset = leaf_disease,
    mask_index = index)

# Display: Button and Interactive Widget
display(interactive_widget)
button_zip_and_save = Button(description="Save and Zip")    # creating the button
button_zip_and_save.on_click(mask.zip)
display(VBox([button_zip_and_save]))

interactive(children=(Dropdown(description='dataset', options=('healthy', 'redrot', 'mosaic', 'rust', 'yellow'…